<a href="https://colab.research.google.com/github/tenslai/mianhuatang002/blob/main/Disco_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⛅**DDv5.61-♣棉花糖AI团队学员版<font size="4">MHT2.0♣**

---



###### <font color="greyblue"> 欢迎您加入棉花糖AI团队，从您打开这里的那一刻起，您已进入神奇的ai绘画世界之旅，愿您在这里，绘制梦想，绘制未来，绘制那些您想象得到或想象不到的所有事、物、景。

###### <font  color="greyblue"> 此版本删减了原版本的第五步及部分多余的部分，让整体操作界面更加精简，并对局部关键部分内容进行注释并加以颜色上的标示，同时，也加入了常用的基础参数设置选项，可进行下拉选择，使其更加便于我们更好地去学习和使用。


##### <font color="black">**特别使用说明：**此版本简化为四个步骤，第一个步骤已经默认设置好，不用设置；第二步骤为模型设置，默认已经关闭二级（次级）模型并默认选择了四个模型组合，此步骤已可满足绝大部分需求，可以选择跳过，不用设置，如需更多模型，可自行选择想要的模型；第三步为基础参数和高级参数设置，里面用<font color="red">**红色标注**</font>的位置参数是需要我们手动修改的，其他无特别颜色标注的请保持默认；第四步出图，除出图张数可以根据需求修改外，其他的可以保持默认。 </font> 

##### ♣♣♣ 
##### <font color="greyblue">*此版本为棉花糖AI团队内部学员使用版本，未经许可不得转载非团队学员使用！* </font> 
##### ♣♣♣ 
<font color="red" size="20">恭喜你，这是一个无效链接，内部学员请找管理重新获得新的地址，感谢配合！**
---


---
#  <font color="orange">**⛅1.准备设置♣**</font><font size="4">（此步骤已设置好，不用设置）

In [ ]:
#@title <font size="3" color="red">不要意思，你现在使用的是非法渠道的链接，你被骗了，快去找卖你链接的人算账，快去哟，晚了来不及了


---
# <font color="orange">**⛅2. 模型设置♣**</font>

In [ ]:
#@title <font size="3" color="red">不要意思，你现在使用的是非法渠道的链接，你被骗了，快去找卖你链接的人算账，快去哟，晚了来不及了

---
# <font color="orange">**⛅3. 基础参数和高级参数♣** </font> <font size="4">（只需修改有红色标注的位置的参数，其他保持默认）</font>

#### <font color="red">**基本参数设置♣**


In [ ]:
#@markdown ---
项目名称 = '不好意思，你不是团队学员，链接未获得许可，我已经搬家' #@param{type: 'string'}



---
# <font color="orange">**⛅4. 关键词出图界面♣**</font> <font size="4">（只需修改有红色标注的位置的参数，其他保持默认）</font>

 <font size="4.8"><font color="red">**关键词♣**


In [ ]:
 不好意思，你不是团队学员，链接未获得许可，我已经搬家

In [ ]:
#@title <font color="red"> 您的图片绘制中，请稍等...
#@markdown ###别等了，快去找卖你链接的人吧，我不会出来的，我搬新家了。


# <font size="3" color="Lightgrey">⭕**温馨提示，项目名称命名的时候，千万不要用特殊符号哟，如/ . @ # %之类的不要出现，否则云盘是建立不了文档的，那么我们出图就不会自动保存了，切记！**</font>